# Load subject data

In [ ]:
import os
import numpy as np
import pandas as pd

# define data root
roamm_root = r"/gpfs1/pi/anon/mindless_reading/ROAMM"
ml_data_root = os.path.join(roamm_root, 'subject_ml_data')
# define subject id
subject_id = 's10014'
subject_dir = os.path.join(ml_data_root, subject_id, 'window_datasets')

# load windowed data and labels
window_size = 256
data_file = os.path.join(subject_dir, f'{subject_id}_{window_size}windowed_data.npy')
label_file = os.path.join(subject_dir, f'{subject_id}_{window_size}windowed_labels.npy')
col_file = os.path.join(subject_dir, f'{subject_id}_col_names.npy')

data = np.load(data_file)
label = np.load(label_file)
col_names = np.load(col_file)

# extract EEG and eye-tracking features
eeg_channel_num = 64
eeg_data = data[:, :eeg_channel_num, :].astype(np.float32)

eye_features = []

In [ ]:
import os
import numpy as np
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler

# define data root
# this is the path to the ROAMM folder on local machine
roamm_root = r"/gpfs1/pi/anon/mindless_reading/ROAMM"
ml_data_root = os.path.join(roamm_root, 'subject_ml_data')
# define subject id
subject_id = 's10014'
subject_dir = os.path.join(ml_data_root, subject_id)
# load all runs for a subject
pkl_files = [f for f in os.listdir(subject_dir) if f.endswith('.pkl')]
df = pd.DataFrame()
for pkl_file in pkl_files:
    df_sub_single_run = pd.read_pickle(os.path.join(subject_dir, pkl_file))
    df_sub_single_run = df_sub_single_run[df_sub_single_run['first_pass_reading'] == 1]
    # convert bool col explicitly to avoid pandas warning
    for col in ['is_blink', 'is_saccade', 'is_fixation', 'is_mw', 'first_pass_reading']:
        df_sub_single_run[col] = df_sub_single_run[col] == True
    df = pd.concat([df, df_sub_single_run])

# normalize pupil size features
df['blink_interp_LPupil_norm'] = df['blink_interp_LPupil'] / df['blink_interp_LPupil'].median()
df['blink_interp_RPupil_norm'] = df['blink_interp_RPupil'] / df['blink_interp_RPupil'].median()

# define EEG and eye tracking features
eeg_cols = df.columns.tolist()[:64]  #first 64 columns are EEG channels
eye_cols = ['blink_interp_LX', 'blink_interp_LY', 'blink_interp_RX', 'blink_interp_RY','blink_interp_LPupil_norm', 'blink_interp_RPupil_norm']

# Downsample data using 1-second windows (fs = 256 Hz)
windowed_data = []
windowed_labels = []
window_size = 64

# Process data in chunks of window_size
for i in range(0, len(df), window_size):
    window = df.iloc[i:i+window_size]
    # Skip if window is too small
    if len(window) < window_size:
        continue
    # Check if labels are consistent in this window
    labels_in_window = window['is_mw'].unique()
    if len(labels_in_window) > 1:
        # Skip windows with mixed labels
        continue

    # Extract features for this window: keep as 2D array (window_size x feature_number)
    eeg_features = window[eeg_cols].values  # shape: (window_size, 64)
    eye_features = window[eye_cols].values  # shape: (window_size, len(eye_cols))
    # Concatenate along the feature axis (axis=1), not the sample axis (axis=0)
    features = np.concatenate([eeg_features, eye_features], axis=1)
    windowed_data.append(features)
    # Use the consistent label
    windowed_labels.append(labels_in_window[0])

# Handle class imbalance
print("Handling class imbalance...")
# Use RandomUnderSampler on flattened data, then recover 3D structure
windowed_data_flat = [w.flatten() for w in windowed_data]
undersampler = RandomUnderSampler(random_state=42)
X_resampled_flat, y_resampled = undersampler.fit_resample(windowed_data_flat, windowed_labels)
# Recover 3D array: (n_samples, window_size, n_features)
window_size = windowed_data[0].shape[0]
n_features = windowed_data[0].shape[1]
X_resampled = np.array(X_resampled_flat).reshape(-1, window_size, n_features)
# print(f"Class distribution:\n{y_resampled.value_counts()}")


# Classifier on raw data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


# First, let's try EEG features only
print("\n" + "="*50)
print("=== EEG Features Only ===")
print("="*50)
X_eeg = X_resampled[eeg_cols].copy()
y_eeg = y_resampled

# Split the data
X_eeg_train, X_eeg_test, y_eeg_train, y_eeg_test = train_test_split(
    X_eeg, y_eeg, test_size=0.2, random_state=42, stratify=y_eeg
)

# Create pipeline with scaling and classifier
eeg_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'))
])

# Train and evaluate EEG-only model
eeg_pipeline.fit(X_eeg_train, y_eeg_train)
y_eeg_pred = eeg_pipeline.predict(X_eeg_test)

print(f"EEG-only Accuracy: {accuracy_score(y_eeg_test, y_eeg_pred):.3f}")
print("\nEEG-only Classification Report:")
print(classification_report(y_eeg_test, y_eeg_pred))
print("\nEEG-only Confusion Matrix:")
print(confusion_matrix(y_eeg_test, y_eeg_pred))

# Next, let's try eye tracking features only
print("\n" + "="*50)
print("=== Eye Tracking Features Only ===")
print("="*50)
X_eye = X_resampled[eye_cols].copy()
y_eye = y_resampled

# Split the data
X_eye_train, X_eye_test, y_eye_train, y_eye_test = train_test_split(
    X_eye, y_eye, test_size=0.2, random_state=42, stratify=y_eye
)

# Create pipeline with scaling and classifier
eye_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'))
])

# Train and evaluate eye-only model
eye_pipeline.fit(X_eye_train, y_eye_train)
y_eye_pred = eye_pipeline.predict(X_eye_test)

print(f"Eye-only Accuracy: {accuracy_score(y_eye_test, y_eye_pred):.3f}")
print("\nEye-only Classification Report:")
print(classification_report(y_eye_test, y_eye_pred))
print("\nEye-only Confusion Matrix:")
print(confusion_matrix(y_eye_test, y_eye_pred))

# Now let's try combined features
print("\n" + "="*50)
print("=== Combined EEG + Eye Tracking Features ===")
print("="*50)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled
)

# Create pipeline with scaling and classifier
combined_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'))
])

# Train and evaluate combined model
combined_pipeline.fit(X_train, y_train)
y_pred = combined_pipeline.predict(X_test)

# Evaluate the model
print(f"Combined Accuracy: {accuracy_score(y_test, y_pred):.3f}")
print("\nCombined Classification Report:")
print(classification_report(y_test, y_pred))
print("\nCombined Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Summary comparison
print("\n" + "="*50)
print("=== PERFORMANCE SUMMARY ===")
print("="*50)
print(f"EEG-only Accuracy:      {accuracy_score(y_eeg_test, y_eeg_pred):.3f}")
print(f"Eye-only Accuracy:      {accuracy_score(y_eye_test, y_eye_pred):.3f}")
print(f"Combined Accuracy:      {accuracy_score(y_test, y_pred):.3f}")


# EEGNet

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
from braindecode.models import EEGNet
from braindecode.util import set_random_seeds

# Prepare EEG data for EEGNet
print("\n" + "="*50)
print("=== EEGNet for Mind-Wandering Classification ===")
print("="*50)

eeg_data = X_resampled[:, :, :64]  
eeg_data = np.transpose(eeg_data, (0, 2, 1))  # (N, num_channels, window_size)
eeg_labels = y_resampled
num_channels = eeg_data.shape[1]
window_size = eeg_data.shape[2]

# Add channel dimension for PyTorch Conv2d
eeg_data_reshaped = eeg_data[:, np.newaxis, :, :]  # (N, 1, window_size, 64)

# Split the data
X_train_eegnet, X_test_eegnet, y_train_eegnet, y_test_eegnet = train_test_split(
    eeg_data_reshaped, eeg_labels, test_size=0.2, random_state=42, stratify=eeg_labels
)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_eegnet)
y_train_tensor = torch.LongTensor(y_train_eegnet)
X_test_tensor = torch.FloatTensor(X_test_eegnet)
y_test_tensor = torch.LongTensor(y_test_eegnet)

# Create data loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Initialize model
# Set up GPU if it is there
cuda = torch.cuda.is_available() 
device = "cuda" if cuda else "cpu"

# Set random seed to be able to reproduce results
seed = 42
set_random_seeds(seed=seed, cuda=cuda)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# define model
model = EEGNet(n_chans=64, n_outputs=2, n_times=window_size)
# print(model)
if cuda:
    model.cuda()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training function
def train_model(model, train_loader, criterion, optimizer, device, epochs=50):
    model.train()
    train_losses = []
    
    for epoch in range(epochs):
        running_loss = 0.0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        avg_loss = running_loss / len(train_loader)
        train_losses.append(avg_loss)
        
        if (epoch + 1) % 10 == 0:
            # Calculate accuracy on training set for this epoch
            model.eval()
            correct = 0
            total = 0
            with torch.no_grad():
                for data, target in train_loader:
                    data, target = data.to(device), target.to(device)
                    outputs = model(data)
                    _, predicted = torch.max(outputs.data, 1)
                    total += target.size(0)
                    correct += (predicted == target).sum().item()
            train_acc = 100 * correct / total
            model.train()
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}, Acc: {train_acc:.2f}%')
    
    return train_losses

# Train the model
print("Training EEGNet...")
train_losses = train_model(model, train_loader, criterion, optimizer, device, epochs=200)

# Evaluation function
def evaluate_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    all_predictions = []
    all_targets = []
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
            
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
    
    accuracy = 100 * correct / total
    return accuracy, all_predictions, all_targets

# Evaluate the model
accuracy, y_pred_eegnet, y_true_eegnet = evaluate_model(model, test_loader, device)

print(f"\nEEGNet Accuracy: {accuracy:.3f}%")
print("\nEEGNet Classification Report:")
print(classification_report(y_true_eegnet, y_pred_eegnet))
print("\nEEGNet Confusion Matrix:")
print(confusion_matrix(y_true_eegnet, y_pred_eegnet))

# Plot training loss
plt.figure(figsize=(10, 6))
plt.plot(train_losses)
plt.title('EEGNet Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.show()

# Updated performance summary
print("\n" + "="*50)
print("=== UPDATED PERFORMANCE SUMMARY ===")
print("="*50)
print(f"EEGNet Accuracy:           {accuracy/100:.3f}")


# ShallowConvNet

In [ ]:
# Define a simple ShallowConvNet architecture for EEG
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

class ShallowConvNet(nn.Module):
    def __init__(self, n_channels, n_samples, n_classes):
        super(ShallowConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 40, (1, 13), padding=(0, 6), bias=False)
        self.conv2 = nn.Conv2d(40, 40, (n_channels, 1), bias=False)
        self.bn1 = nn.BatchNorm2d(40)
        self.pool = nn.AvgPool2d(kernel_size=(1, 35), stride=(1, 7))
        self.dropout = nn.Dropout(0.5)
        self.elu = nn.ELU()
        self._n_samples = n_samples
        self.fc = nn.Linear(self._get_flattened_size(n_channels, n_samples), n_classes)

    def _get_flattened_size(self, n_channels, n_samples):
        # Calculate the size after conv/pool for the FC layer
        with torch.no_grad():
            x = torch.zeros(1, 1, n_channels, n_samples)
            x = self.conv1(x)
            x = self.conv2(x)
            x = self.bn1(x)
            x = self.elu(x)
            x = self.pool(x)
            x = self.dropout(x)
            return x.view(1, -1).shape[1]

    def forward(self, x):
        # x shape: (batch, 1, n_channels, n_samples)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.bn1(x)
        x = self.elu(x)
        x = self.pool(x)
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Set parameters
window_size = 256
num_channels = 64
num_classes = len(set(y_resampled))
batch_size = 32
learning_rate = 0.001
epochs = 100

# Prepare EEG data (ensure correct shape: samples, 1, channels, samples)
eeg_data = X_resampled[:, :, :num_channels]  # shape: (N, window_size, num_channels)
eeg_labels = y_resampled

# Convert to torch tensors and reshape
eeg_data_tensor = torch.tensor(eeg_data, dtype=torch.float32).permute(0, 2, 1).unsqueeze(1)  # (N, 1, C, T)
eeg_labels_tensor = torch.tensor(eeg_labels, dtype=torch.long)

# Split into train/test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    eeg_data_tensor, eeg_labels_tensor, test_size=0.2, random_state=42, stratify=eeg_labels_tensor
)

# Create DataLoaders
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Instantiate model, loss, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ShallowConvNet(num_channels, window_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
train_losses = []
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * data.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    train_losses.append(epoch_loss)
    if (epoch+1) % 10 == 0 or epoch == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")

# Evaluate the model
def evaluate_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    all_predictions = []
    all_targets = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
    accuracy = 100 * correct / total
    return accuracy, all_predictions, all_targets

accuracy, y_pred_shallow, y_true_shallow = evaluate_model(model, test_loader, device)

print(f"\nShallowConvNet Accuracy: {accuracy:.3f}%")
from sklearn.metrics import classification_report, confusion_matrix
print("\nShallowConvNet Classification Report:")
print(classification_report(y_true_shallow, y_pred_shallow))
print("\nShallowConvNet Confusion Matrix:")
print(confusion_matrix(y_true_shallow, y_pred_shallow))

import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.plot(train_losses)
plt.title('ShallowConvNet Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.show()

print("\n" + "="*50)
print("=== SHALLOW CONVNET PERFORMANCE SUMMARY ===")
print("="*50)
print(f"ShallowConvNet Accuracy:           {accuracy/100:.3f}")


In [ ]:
import torch
import numpy as np
from braindecode.models import EEGNet
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import torch.nn as nn
import matplotlib.pyplot as plt

# Prepare EEGNet data
# Assume X_resampled: (N, window_size, num_channels), y_resampled: (N,)
# EEGNet expects input shape: (batch, 1, n_chans, n_times)
# So we need to transpose and add channel dimension

# Set EEGNet parameters based on your data
# Transpose to (N, n_chans, n_times)
num_channels = 64
eeg_data = X_resampled[:, :, :num_channels]  # (N, window_size, num_channels)
eeg_data = np.transpose(eeg_data, (0, 2, 1))  # (N, num_channels, window_size)
eeg_labels = y_resampled

# Add channel dimension for PyTorch Conv2d: (N, 1, n_chans, n_times)
eeg_data_reshaped = eeg_data[:, np.newaxis, :, :]  # (N, 1, n_chans, n_times)

# Split the data
X_train_eegnet, X_test_eegnet, y_train_eegnet, y_test_eegnet = train_test_split(
    eeg_data_reshaped, eeg_labels, test_size=0.2, random_state=42, stratify=eeg_labels
)

# Convert to torch tensors
X_train_eegnet = torch.tensor(X_train_eegnet, dtype=torch.float32)
X_test_eegnet = torch.tensor(X_test_eegnet, dtype=torch.float32)
y_train_eegnet = torch.tensor(y_train_eegnet, dtype=torch.long)
y_test_eegnet = torch.tensor(y_test_eegnet, dtype=torch.long)

# DataLoader
batch_size = 32
train_dataset_eegnet = TensorDataset(X_train_eegnet, y_train_eegnet)
test_dataset_eegnet = TensorDataset(X_test_eegnet, y_test_eegnet)
train_loader_eegnet = DataLoader(train_dataset_eegnet, batch_size=batch_size, shuffle=True)
test_loader_eegnet = DataLoader(test_dataset_eegnet, batch_size=batch_size, shuffle=False)

# Set model parameters
n_chans = X_train_eegnet.shape[2]  # number of EEG channels
n_times = X_train_eegnet.shape[3]  # number of time samples
n_outputs = len(np.unique(y_train_eegnet.numpy()))  # number of classes

# Instantiate EEGNet model
eegnet_model = EEGNet(
    n_chans=n_chans,
    n_outputs=n_outputs,
    n_times=n_times,
    F1=8,
    D=2,
    F2=None,  # default is F1*D
    kernel_length=64,
    depthwise_kernel_length=16,
    pool1_kernel_size=4,
    pool2_kernel_size=8,
    final_conv_length="auto",
    drop_prob=0.25,
    activation=nn.ELU,
    batch_norm_momentum=0.01,
    batch_norm_affine=True,
    batch_norm_eps=1e-3,
    conv_spatial_max_norm=1,
    final_layer_with_constraint=False,
    norm_rate=0.25,
)

eegnet_model = eegnet_model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(eegnet_model.parameters(), lr=0.001)

# Train the model
def train_model_eegnet(model, train_loader, criterion, optimizer, device, epochs=200):
    model.train()
    train_losses = []
    for epoch in range(epochs):
        epoch_loss = 0.0
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * data.size(0)
        epoch_loss /= len(train_loader.dataset)
        train_losses.append(epoch_loss)
        if (epoch+1) % 10 == 0 or epoch == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")
    return train_losses

print("Training EEGNet...")
eegnet_train_losses = train_model_eegnet(eegnet_model, train_loader_eegnet, criterion, optimizer, device, epochs=200)

# Evaluate the model
def evaluate_model_eegnet(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    all_predictions = []
    all_targets = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
    accuracy = 100 * correct / total
    return accuracy, all_predictions, all_targets

eegnet_accuracy, y_pred_eegnet, y_true_eegnet = evaluate_model_eegnet(eegnet_model, test_loader_eegnet, device)

print(f"\nEEGNet Accuracy: {eegnet_accuracy:.3f}%")
print("\nEEGNet Classification Report:")
print(classification_report(y_true_eegnet, y_pred_eegnet))
print("\nEEGNet Confusion Matrix:")
print(confusion_matrix(y_true_eegnet, y_pred_eegnet))

plt.figure(figsize=(10, 6))
plt.plot(eegnet_train_losses)
plt.title('EEGNet Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.show()

print("\n" + "="*50)
print("=== EEGNET PERFORMANCE SUMMARY ===")
print("="*50)
print(f"EEGNet Accuracy:           {eegnet_accuracy/100:.3f}")


In [ ]:
from braindecode.datasets import MOABBDataset

subject_id = 3
# Load the dataset
dataset = BNCI2014_001()

import numpy as np

from braindecode.preprocessing import (
    Preprocessor,
    exponential_moving_standardize,
    preprocess,
)

low_cut_hz = 4.0  # low cut frequency for filtering
high_cut_hz = 38.0  # high cut frequency for filtering
# Parameters for exponential moving standardization
factor_new = 1e-3
init_block_size = 1000

preprocessors = [
    Preprocessor("pick_types", eeg=True, meg=False, stim=False),  # Keep EEG sensors
    Preprocessor(
        lambda data, factor: np.multiply(data, factor),  # Convert from V to uV
        factor=1e6,
    ),
    Preprocessor("filter", l_freq=low_cut_hz, h_freq=high_cut_hz),  # Bandpass filter
    Preprocessor(
        exponential_moving_standardize,  # Exponential moving standardization
        factor_new=factor_new,
        init_block_size=init_block_size,
    ),
]

# Preprocess the data
preprocess(dataset, preprocessors, n_jobs=-1)

from braindecode.preprocessing import create_windows_from_events

trial_start_offset_seconds = -0.5
# Extract sampling frequency, check that they are same in all datasets
sfreq = dataset.datasets[0].raw.info["sfreq"]
assert all([ds.raw.info["sfreq"] == sfreq for ds in dataset.datasets])
# Calculate the window start offset in samples.
trial_start_offset_samples = int(trial_start_offset_seconds * sfreq)

# Create windows using braindecode function for this. It needs parameters to
# define how windows should be used.
windows_dataset = create_windows_from_events(
    dataset,
    trial_start_offset_samples=trial_start_offset_samples,
    trial_stop_offset_samples=0,
    preload=True,
)

splitted = windows_dataset.split("session")
train_set = splitted["0train"]  # Session train
test_set = splitted["1test"]  # Session evaluation

# EEGNet

In [ ]:
from codecs import raw_unicode_escape_decode
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
import torch
import mne
from sklearn.pipeline import make_pipeline
from skorch.callbacks import EarlyStopping, EpochScoring
from skorch.dataset import ValidSplit
from braindecode import EEGClassifier
from braindecode.models import EEGNet
from braindecode.util import set_random_seeds
from skorch.callbacks import LRScheduler
from braindecode.datasets import BaseConcatDataset, BaseDataset
from braindecode.preprocessing import create_fixed_length_windows


# Transpose to (N, n_chans, n_times)
num_channels = 64
eeg_data = X_resampled[:, :, :num_channels]  # (N, window_size, num_channels)
eeg_data = np.transpose(eeg_data, (0, 2, 1))  # (N, num_channels, window_size)
eeg_labels = np.array(y_resampled, dtype=int)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    eeg_data, eeg_labels, test_size=0.2, random_state=42, stratify=eeg_labels
)

# Create datasets
sampling_freq = 256  # in Hertz
info = mne.create_info(num_channels, sfreq=sampling_freq)
datasets = []
for i in range(len(X_train)):
    raw = mne.io.RawArray(X_train[i], info, verbose=False)
    description = pd.Series(data=[y_train[i], 'train'], index=["target", "session"])
    datasets.append(BaseDataset(raw, description, target_name="target"))

for j in range(len(X_test)):
    raw = mne.io.RawArray(X_test[j], info, verbose=False)
    description = pd.Series(data=[y_test[j], 'test'], index=["target", "session"])
    datasets.append(BaseDataset(raw, description, target_name="target"))
datasets = BaseConcatDataset(datasets)

# Set up GPU if it is there
cuda = torch.cuda.is_available() 
device = "cuda" if cuda else "cpu"

# Set random seed to be able to reproduce results
seed = 42
set_random_seeds(seed=seed, cuda=cuda)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# define model
model = EEGNet(n_chans=64, n_outputs=2, n_times=window_size)
# print(model)
if cuda:
    model.cuda()

# Splitting windowed data into train, valid and test subsets.
splits = datasets.split("session")
train_set = splits["train"]
# valid_set = splits["valid"]
test_set = splits["test"]

# define training hyperparameters
lr = 0.0625 * 0.1
weight_decay = 0
batch_size = 32
n_epochs = 30
patience = 3

clf = EEGClassifier(
    model,
    criterion=torch.nn.CrossEntropyLoss,
    optimizer=torch.optim.Adam,
    train_split=None,
    optimizer__lr=lr,
    optimizer__weight_decay=weight_decay,
    batch_size=batch_size,
    callbacks=[
        "accuracy",
        ("lr_scheduler", LRScheduler("CosineAnnealingLR", T_max=n_epochs - 1)),
    ],
    verbose=1,  # Not printing the results for each epoch
    device=device,
    classes=list(range(2)),
    max_epochs=n_epochs,
)

# Model training for a specified number of epochs. `y` is None as it is already supplied
# in the dataset.
# clf.fit(train_set, y=None)

# # evaluated the model after training
y_test_ = test_set.get_metadata().target
# test_acc = clf.score(test_set, y=y_test)
# print(f"Test acc: {(test_acc * 100):.2f}%")



# Band Power Analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import pandas as pd

eeg_data = X_resampled[:, :, :64]  
eeg_data = np.transpose(eeg_data, (0, 2, 1))  # (N, num_channels, window_size)

def compute_band_power(data, fs=256, window_size=64):
    """
    Compute band power for different frequency bands over 64-sample windows.
    
    Parameters:
    -----------
    data : array-like, shape (n_samples, n_channels) or (n_channels, n_samples)
        EEG data
    fs : int, default=256
        Sampling frequency in Hz
    window_size : int, default=64
        Window size for analysis
        
    Returns:
    --------
    band_powers : dict
        Dictionary containing power for each frequency band
    """
    
    # Define frequency bands (in Hz)
    bands = {
        'delta': (1, 4),
        'theta': (4, 8), 
        'alpha': (8, 13),
        'beta': (13, 30),
        'gamma': (30, 50)
    }
    
    # Ensure data is in the right format (samples, channels)
    if data.shape[0] < data.shape[1]:
        data = data.T
    
    n_samples, n_channels = data.shape
    n_windows = n_samples // window_size
    
    # Initialize results dictionary
    band_powers = {band: np.zeros((n_windows, n_channels)) for band in bands.keys()}
    
    for i in range(n_windows):
        start_idx = i * window_size
        end_idx = start_idx + window_size
        window_data = data[start_idx:end_idx, :]
        
        for ch in range(n_channels):
            # Compute power spectral density using Welch's method
            freqs, psd = signal.welch(window_data[:, ch], fs=fs, nperseg=window_size, 
                                    noverlap=window_size//2, nfft=window_size)
            
            # Compute band power for each frequency band
            for band_name, (low_freq, high_freq) in bands.items():
                # Find frequency indices for the band
                freq_mask = (freqs >= low_freq) & (freqs <= high_freq)
                # Compute power in the band (integrate PSD)
                band_power = np.trapz(psd[freq_mask], freqs[freq_mask])
                band_powers[band_name][i, ch] = band_power
    
    return band_powers, freqs, n_windows

# Test with the existing EEG data
print("Computing band powers for EEG data...")
print(f"EEG data shape: {X_resampled.shape}")

# Extract EEG data (first 64 channels)
eeg_data = X_resampled[:, :, :64]  # Shape: (n_samples, window_size, n_channels)

# Take first sample for demonstration
sample_eeg = eeg_data[0]  # Shape: (window_size, n_channels)
print(f"Sample EEG shape: {sample_eeg.shape}")

# Compute band powers
band_powers, freqs, n_windows = compute_band_power(sample_eeg, fs=256, window_size=64)

print(f"\nComputed band powers for {n_windows} windows")
print("Band power shapes:")
for band_name, powers in band_powers.items():
    print(f"  {band_name}: {powers.shape}")
    print(f"  {band_name} mean power: {np.mean(powers):.6f}")


In [ ]:
# Compute band powers for multiple samples and visualize results
print("Computing band powers for multiple EEG samples...")

# Compute band powers for first 10 samples
n_samples_to_analyze = min(10, len(eeg_data))
all_band_powers = {band: [] for band in ['delta', 'theta', 'alpha', 'beta', 'gamma']}

for i in range(n_samples_to_analyze):
    sample_eeg = eeg_data[i]  # Shape: (window_size, n_channels)
    band_powers, freqs, n_windows = compute_band_power(sample_eeg, fs=256, window_size=64)
    
    # Average across channels and windows for this sample
    for band_name, powers in band_powers.items():
        mean_power = np.mean(powers)
        all_band_powers[band_name].append(mean_power)

# Create DataFrame for easy analysis
band_power_df = pd.DataFrame(all_band_powers)
band_power_df.index.name = 'Sample'

print("\nBand power statistics across samples:")
print(band_power_df.describe())

# Visualize band powers
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Band power distribution
axes[0, 0].boxplot([all_band_powers[band] for band in all_band_powers.keys()], 
                   labels=list(all_band_powers.keys()))
axes[0, 0].set_title('Band Power Distribution Across Samples')
axes[0, 0].set_ylabel('Power (μV²/Hz)')
axes[0, 0].tick_params(axis='x', rotation=45)

# Plot 2: Band power across samples
for band_name, powers in all_band_powers.items():
    axes[0, 1].plot(powers, marker='o', label=band_name)
axes[0, 1].set_title('Band Power Across Samples')
axes[0, 1].set_xlabel('Sample Index')
axes[0, 1].set_ylabel('Power (μV²/Hz)')
axes[0, 1].legend()
axes[0, 1].grid(True)

# Plot 3: Relative band power (normalized)
normalized_powers = band_power_df.div(band_power_df.sum(axis=1), axis=0)
normalized_powers.plot(kind='bar', stacked=True, ax=axes[1, 0])
axes[1, 0].set_title('Relative Band Power (Normalized)')
axes[1, 0].set_xlabel('Sample Index')
axes[1, 0].set_ylabel('Relative Power')
axes[1, 0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Plot 4: Power spectral density for one sample
sample_eeg = eeg_data[0]  # First sample
# Average across channels
avg_signal = np.mean(sample_eeg, axis=1)
freqs_psd, psd = signal.welch(avg_signal, fs=256, nperseg=64, noverlap=32, nfft=64)

axes[1, 1].semilogy(freqs_psd, psd)
axes[1, 1].set_title('Power Spectral Density (Sample 0, Averaged Across Channels)')
axes[1, 1].set_xlabel('Frequency (Hz)')
axes[1, 1].set_ylabel('Power Spectral Density (μV²/Hz)')
axes[1, 1].grid(True)

# Add vertical lines for band boundaries
band_boundaries = [1, 4, 8, 13, 30, 50]
colors = ['red', 'orange', 'green', 'blue', 'purple']
for i, freq in enumerate(band_boundaries[:-1]):
    axes[1, 1].axvline(freq, color=colors[i], linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

print(f"\nAnalyzed {n_samples_to_analyze} samples with {window_size}-sample windows")
print("Band frequency ranges:")
bands = {'delta': (1, 4), 'theta': (4, 8), 'alpha': (8, 13), 'beta': (13, 30), 'gamma': (30, 50)}
for band_name, (low, high) in bands.items():
    print(f"  {band_name}: {low}-{high} Hz")


In [ ]:
# Create band power features for machine learning
def extract_band_power_features(eeg_data, fs=256, window_size=64):
    """
    Extract band power features from EEG data for machine learning.
    
    Parameters:
    -----------
    eeg_data : array-like, shape (n_samples, window_size, n_channels)
        EEG data
    fs : int, default=256
        Sampling frequency in Hz
    window_size : int, default=64
        Window size for analysis
        
    Returns:
    --------
    features : array, shape (n_samples, n_features)
        Band power features for each sample
    feature_names : list
        Names of the features
    """
    
    n_samples, _, n_channels = eeg_data.shape
    bands = ['delta', 'theta', 'alpha', 'beta', 'gamma']
    
    # Initialize feature matrix
    # Features: [mean_power_per_band_per_channel, relative_power_per_band, total_power_per_channel]
    n_features = len(bands) * n_channels + len(bands) + n_channels
    features = np.zeros((n_samples, n_features))
    
    # Create feature names
    feature_names = []
    
    # Band power per channel features
    for band in bands:
        for ch in range(n_channels):
            feature_names.append(f'{band}_power_ch{ch:02d}')
    
    # Relative band power features (summed across channels)
    for band in bands:
        feature_names.append(f'{band}_relative_power')
    
    # Total power per channel
    for ch in range(n_channels):
        feature_names.append(f'total_power_ch{ch:02d}')
    
    for i in range(n_samples):
        sample_eeg = eeg_data[i]  # Shape: (window_size, n_channels)
        band_powers, _, _ = compute_band_power(sample_eeg, fs=fs, window_size=window_size)
        
        feature_idx = 0
        
        # Band power per channel
        for band in bands:
            powers = band_powers[band]  # Shape: (n_windows, n_channels)
            mean_powers = np.mean(powers, axis=0)  # Average across windows
            features[i, feature_idx:feature_idx+n_channels] = mean_powers
            feature_idx += n_channels
        
        # Relative band power (summed across channels and normalized)
        total_powers = []
        for band in bands:
            powers = band_powers[band]
            total_power = np.sum(np.mean(powers, axis=0))  # Sum across channels, avg across windows
            total_powers.append(total_power)
        
        # Normalize to get relative powers
        total_sum = sum(total_powers)
        if total_sum > 0:
            relative_powers = [p / total_sum for p in total_powers]
        else:
            relative_powers = [0] * len(bands)
        
        features[i, feature_idx:feature_idx+len(bands)] = relative_powers
        feature_idx += len(bands)
        
        # Total power per channel
        channel_totals = np.zeros(n_channels)
        for band in bands:
            powers = band_powers[band]
            channel_totals += np.mean(powers, axis=0)
        
        features[i, feature_idx:feature_idx+n_channels] = channel_totals
    
    return features, feature_names

# Extract band power features for the entire dataset
print("Extracting band power features for machine learning...")
print(f"Processing {len(eeg_data)} samples...")

band_power_features, feature_names = extract_band_power_features(eeg_data, fs=256, window_size=64)

print(f"\nExtracted features shape: {band_power_features.shape}")
print(f"Number of features: {len(feature_names)}")
print(f"Feature categories:")
print(f"  - Band power per channel: {5 * 64} features")
print(f"  - Relative band power: {5} features") 
print(f"  - Total power per channel: {64} features")

# Show some example features
print(f"\nFirst 10 feature names:")
for i, name in enumerate(feature_names[:10]):
    print(f"  {i}: {name}")

print(f"\nLast 10 feature names:")
for i, name in enumerate(feature_names[-10:], len(feature_names)-10):
    print(f"  {i}: {name}")

# Basic statistics
print(f"\nFeature statistics:")
print(f"  Mean: {np.mean(band_power_features):.6f}")
print(f"  Std: {np.std(band_power_features):.6f}")
print(f"  Min: {np.min(band_power_features):.6f}")
print(f"  Max: {np.max(band_power_features):.6f}")

# Visualize feature distributions for first few features
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for i in range(6):
    axes[i].hist(band_power_features[:, i], bins=30, alpha=0.7)
    axes[i].set_title(f'{feature_names[i]}')
    axes[i].set_xlabel('Power')
    axes[i].set_ylabel('Frequency')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nBand power features extracted successfully!")
print(f"These features can be used as input to machine learning models.")
print(f"Shape: (n_samples={band_power_features.shape[0]}, n_features={band_power_features.shape[1]})")


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

eeg_labels = np.array(y_resampled, dtype=int)
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    band_power_features, eeg_labels, test_size=0.2, random_state=42, stratify=eeg_labels
)

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "SVM (RBF kernel)": SVC(kernel='rbf', random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "Naive Bayes": GaussianNB()
}

results = {}

# You can adjust n_components for PCA as needed (e.g., 0.95 for 95% variance, or an int)
pca_n_components = 0.95

for name, model in models.items():
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=pca_n_components, random_state=42)),
        ('clf', model)
    ])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc
    print(f"\n{name} Accuracy: {acc:.4f}")
    print(classification_report(y_test, y_pred))

import pandas as pd
results_df = pd.DataFrame(list(results.items()), columns=["Model", "Accuracy"])
print("\nSummary of model accuracies:")
print(results_df)

# Create Braindecode datasets

In [ ]:
import os
import numpy as np
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
from braindecode.datasets import create_from_X_y
from sklearn.model_selection import train_test_split

# define data root
# this is the path to the ROAMM folder on local machine
roamm_root = r"/gpfs1/pi/anon/mindless_reading/ROAMM"
ml_data_root = os.path.join(roamm_root, 'subject_ml_data')
# define subject id
subject_id = 's10014'
subject_dir = os.path.join(ml_data_root, subject_id)
# load all runs for a subject
pkl_files = [f for f in os.listdir(subject_dir) if f.endswith('.pkl')]
df = pd.DataFrame()
for pkl_file in pkl_files:
    df_sub_single_run = pd.read_pickle(os.path.join(subject_dir, pkl_file))
    df_sub_single_run = df_sub_single_run[df_sub_single_run['first_pass_reading'] == 1]
    # convert bool col explicitly to avoid pandas warning
    for col in ['is_blink', 'is_saccade', 'is_fixation', 'is_mw', 'first_pass_reading']:
        df_sub_single_run[col] = df_sub_single_run[col] == True
    df = pd.concat([df, df_sub_single_run])

# normalize pupil size features
df['blink_interp_LPupil_norm'] = df['blink_interp_LPupil'] / df['blink_interp_LPupil'].median()
df['blink_interp_RPupil_norm'] = df['blink_interp_RPupil'] / df['blink_interp_RPupil'].median()

# define EEG and eye tracking features
ch_names = df.columns.tolist()[:64]  #first 64 columns are EEG channels
sfreq = 256
window_seconds = 0.5
# Downsample data using 1-second windows (fs = 256 Hz)
windowed_data = []
windowed_labels = []
window_size = int(sfreq * window_seconds)

# Process data in chunks of window_size
for i in range(0, len(df), window_size):
    window = df.iloc[i:i+window_size]
    # Skip if window is too small
    if len(window) < window_size:
        continue
    # Check if labels are consistent in this window
    labels_in_window = window['is_mw'].unique()
    if len(labels_in_window) > 1:
        # Skip windows with mixed labels
        continue

    # Extract features for this window: keep as 2D array (window_size x feature_number)
    eeg_data = window[ch_names].values
    windowed_data.append(eeg_data)
    # Use the consistent label
    windowed_labels.append(labels_in_window[0])

# Use RandomUnderSampler on flattened data, then recover 3D structure
windowed_data_flat = [w.flatten() for w in windowed_data]
undersampler = RandomUnderSampler(random_state=42)
X_resampled_flat, y_resampled = undersampler.fit_resample(windowed_data_flat, windowed_labels)
# Recover 3D array: (n_samples, window_size, n_features)
window_size = windowed_data[0].shape[0]
n_features = windowed_data[0].shape[1]
X_resampled = np.array(X_resampled_flat).reshape(-1, window_size, n_features)
X = np.transpose(X_resampled, (0, 2, 1))  # (N, num_channels, window_size)
y = np.array(y_resampled, dtype=int)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# create braindecode datasets for training and testing
train_set = create_from_X_y(
    X_train,
    y_train,
    drop_last_window=False,
    sfreq=sfreq,
    ch_names=ch_names,
    window_stride_samples=window_size,
    window_size_samples=window_size
)

test_set = create_from_X_y(
    X_test,
    y_test,
    drop_last_window=False,
    sfreq=sfreq,
    ch_names=ch_names,
    window_stride_samples=window_size,
    window_size_samples=window_size
)



# Call EEGNet

In [ ]:
from codecs import raw_unicode_escape_decode
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from sklearn.pipeline import make_pipeline
from skorch.callbacks import EarlyStopping, EpochScoring
from skorch.dataset import ValidSplit
from braindecode import EEGClassifier
from braindecode.models import EEGNet, ATCNet
from braindecode.util import set_random_seeds
from skorch.callbacks import LRScheduler
from braindecode.datasets import BaseConcatDataset, BaseDataset
from braindecode.preprocessing import create_fixed_length_windows

# Set up GPU if it is there
cuda = torch.cuda.is_available() 
device = "cuda" if cuda else "cpu"

# Set random seed to be able to reproduce results
seed = 42
set_random_seeds(seed=seed, cuda=cuda)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# define model
model = EEGNet(n_chans=64, n_outputs=2, n_times=window_size)
# model = ATCNet(n_chans=64, n_outputs=2, input_window_seconds=window_seconds, sfreq=sfreq, chs_info=ch_names, n_times=window_size)
# print(model)
if cuda:
    model.cuda()

# define training hyperparameters
lr = 0.0625 * 0.1
weight_decay = 0
batch_size = 32
n_epochs = 100
patience = 3

clf = EEGClassifier(
    model,
    criterion=torch.nn.CrossEntropyLoss,
    optimizer=torch.optim.Adam,
    train_split=None,
    optimizer__lr=lr,
    optimizer__weight_decay=weight_decay,
    batch_size=batch_size,
    callbacks=[
        "accuracy",
        ("lr_scheduler", LRScheduler("CosineAnnealingLR", T_max=n_epochs - 1)),
    ],
    verbose=1,  # Not printing the results for each epoch
    device=device,
    classes=list(range(2)),
    max_epochs=n_epochs,
)

# Model training for a specified number of epochs. `y` is None as it is already supplied
# in the dataset.
clf.fit(train_set, y=None)

from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay, accuracy_score, f1_score
import matplotlib.pyplot as plt

# Predict labels
y_pred = clf.predict(test_set)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
fig, ax = plt.subplots(figsize=(6, 6))
disp.plot(ax=ax, cmap='Blues', colorbar=False)
plt.title("Confusion Matrix")
plt.show()

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=4))

# Print additional metrics
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Accuracy: {acc:.4f}")
print(f"Weighted F1 Score: {f1:.4f}")


# PSD

In [ ]:
import os
import numpy as np
import mne

# define data root
# this is the path to the ROAMM folder on local machine
roamm_root = r"/gpfs1/pi/anon/mindless_reading/ROAMM"
ml_data_root = os.path.join(roamm_root, 'subject_ml_data')
# define subject id
subject_id = 's10014'
subject_dir = os.path.join(ml_data_root, subject_id, 'window_datasets')
# load single data
data = np.load(os.path.join(subject_dir, 's10014_256windowed_data.npy'))
labels = np.load(os.path.join(subject_dir, 's10014_256windowed_labels.npy'))
col_name = np.load(os.path.join(subject_dir, 's10014_col_names.npy'))
sfreq = 256

# get the 64-channel eeg data (first 64 cols)
eeg_data = data[:, :64, :]
eeg_data = eeg_data.astype(np.float64)
psds, freqs = mne.time_frequency.psd_array_multitaper(eeg_data, sfreq, fmax=50, remove_dc=False)



In [ ]:
import matplotlib.pyplot as plt

# plot each channel PSD for on-task and mind-wandering
labels = labels.astype(bool)
psds_ot = np.mean(psds[~labels], axis=0)
psds_mw = np.mean(psds[labels], axis=0)

# Convert to dB (decibels)
# psds_ot_db = 10 * np.log10(psds_ot)
# psds_mw_db = 10 * np.log10(psds_mw)

print("Creating individual channel plots...")

# Create the plot with proper indexing
plt.figure(figsize=(10, 10))

# Plot on-task condition (all 64 channels)
plt.subplot(2, 1, 1)
# for ch_idx in range(psds_ot_db.shape[0]):  # Iterate over channels
#     plt.plot(freqs, psds_ot_db[ch_idx, :], alpha=0.7, linewidth=0.8)

for ch_idx in range(psds_ot.shape[0]):  # Iterate over channels
    plt.plot(freqs, psds_ot[ch_idx, :], alpha=0.7, linewidth=0.8)

plt.title('On-Task: Power Spectral Density for All 64 EEG Channels', fontsize=14)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power (dB)')
plt.grid(True, alpha=0.3)
plt.xlim(0, 50)

# Plot mind-wandering condition (all 64 channels)
plt.subplot(2, 1, 2)
# for ch_idx in range(psds_mw_db.shape[0]):  # Iterate over channels
#     plt.plot(freqs, psds_mw_db[ch_idx, :], alpha=0.7, linewidth=0.8)

for ch_idx in range(psds_mw.shape[0]):  # Iterate over channels
    plt.plot(freqs, psds_mw[ch_idx, :], alpha=0.7, linewidth=0.8)

plt.title('Mind-Wandering: Power Spectral Density for All 64 EEG Channels', fontsize=14)
plt.xlabel('Frequency (Hz)')
# plt.ylabel('Power (dB)')
plt.ylabel('Power (μV²/Hz)')
plt.grid(True, alpha=0.3)
plt.xlim(0, 50)

plt.tight_layout()
plt.show()

             

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "SVM (RBF kernel)": SVC(kernel='rbf', random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "Naive Bayes": GaussianNB()
}

results = {}

# You can adjust n_components for PCA as needed (e.g., 0.95 for 95% variance, or an int)
pca_n_components = 0.95

# Split the data
psds_flattened = psds.reshape(psds.shape[0], -1)
# convert to dB
psds_flattened = 10 * np.log10(psds_flattened)
X_train, X_test, y_train, y_test = train_test_split(
    psds_flattened, labels, test_size=0.2, random_state=42, stratify=labels
)

for name, model in models.items():
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=pca_n_components, random_state=42)),
        ('clf', model)
    ])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc
    print(f"\n{name} Accuracy: {acc:.4f}")
    print(classification_report(y_test, y_pred))

import pandas as pd
results_df = pd.DataFrame(list(results.items()), columns=["Model", "Accuracy"])
print("\nSummary of model accuracies:")
print(results_df)

# Generate feature files

In [ ]:
import os
import mne
import numpy as np
import pandas as pd

# ==========================
# Config
# ==========================
data_root = "/gpfs1/pi/anon/mindless_reading/data"
sfreq = 256
window_seconds = 2
window_size = int(sfreq * window_seconds)

# ==========================
# EEG band definitions (global)
# ==========================
band_names = [
    "theta1", "theta2", "alpha1", "alpha2",
    "beta1", "beta2", "gamma1", "gamma2",
]

band_defs = [
    (4.0, 6.0),    # theta1
    (6.5, 8.0),    # theta2
    (8.5, 10.0),   # alpha1
    (10.5, 13.0),  # alpha2
    (13.5, 18.0),  # beta1
    (18.5, 30.0),  # beta2
    (30.5, 40.0),  # gamma1
    (40.0, 49.5),  # gamma2
]

n_bands = len(band_defs)

# ==========================
# Helper functions
# ==========================
def get_col_array(data, col_names, targets):
    """
    Return array shaped (n_epochs, seq_len) for the first matching column
    in `targets`. If column not found, return None.
    """
    idx = np.where(np.isin(col_names, targets))[0]
    if idx.size == 0:
        return None

    arr = data[:, idx].astype(np.float64)  # (n_epochs, 1, seq_len) typically
    if arr.ndim == 3 and arr.shape[1] == 1:
        arr = arr[:, 0, :]  # (n_epochs, seq_len)
    return arr


def mean_per_epoch(arr):
    """
    Compute per-epoch mean across valid (non-NaN) timepoints.
    Returns list of length n_epochs; NaN if no valid points.
    """
    out = []
    mask = ~np.isnan(arr)
    for row, m in zip(arr, mask):
        if m.any():
            out.append(np.nanmean(row[m]))
        else:
            out.append(np.nan)
    return out


def unique_counts_per_epoch(arr):
    """
    Count unique valid values per epoch (0 if none).
    """
    out = []
    mask = ~np.isnan(arr)
    for row, m in zip(arr, mask):
        if m.any():
            out.append(np.unique(row[m]).size)
        else:
            out.append(0)
    return out


# ==========================
# Collect subjects
# ==========================
all_subjects = sorted(
    d for d in os.listdir(data_root)
    if d.startswith("s") and os.path.isdir(os.path.join(data_root, d))
)

epoch_counts = []

# ==========================
# Main loop over subjects
# ==========================
for subject_id in all_subjects:
    subject_dir = os.path.join(
        data_root,
        subject_id,
        "ml_data",
        f"{window_size}window_datasets",
    )

    data_file = os.path.join(subject_dir, f"{subject_id}_{window_size}windowed_data.npy")
    label_file = os.path.join(subject_dir, f"{subject_id}_{window_size}windowed_labels.npy")
    col_file = os.path.join(subject_dir, f"{subject_id}_col_names.npy")

    # Skip subjects with missing files
    if not (os.path.exists(data_file) and os.path.exists(label_file) and os.path.exists(col_file)):
        print(f"Missing files for {subject_id}, skipping.")
        epoch_counts.append(0)
        continue

    data = np.load(data_file)
    labels = np.load(label_file)
    col_name = np.load(col_file)

    epoch_count = data.shape[0]
    epoch_counts.append(epoch_count)

    # check sample count for every subject
    if epoch_count < 10:
        print(f"Subject {subject_id} has insufficient data samples: {epoch_count} samples.")
        continue

    # ==========================
    # EEG PSD features
    # ==========================
    # get the 64-channel EEG data (first 64 cols)
    eeg_data = data[:, :64, :].astype(np.float64)

    psds, freqs = mne.time_frequency.psd_array_multitaper(
        eeg_data,
        sfreq,
        fmin=4,
        fmax=50,
        output="power",
    )

    n_epochs, n_ch, _ = psds.shape

    psds_band = np.empty((n_epochs, n_ch, n_bands), dtype=psds.dtype)
    for i, (fmin, fmax) in enumerate(band_defs):
        idx = (freqs >= fmin) & (freqs <= fmax)
        psds_band[:, :, i] = psds[:, :, idx].mean(axis=-1)

    # Flatten features for ML: (n_epochs, n_ch * n_bands)
    psds_band_flat = psds_band.reshape(n_epochs, -1)

    # column names: chan_band
    eeg_columns = [
        f"{ch}_{band}"
        for ch in col_name[:n_ch]
        for band in band_names
    ]
    df_eeg = pd.DataFrame(psds_band_flat, columns=eeg_columns)

    # ==========================
    # Eye-tracking features
    # ==========================
    df_eye = pd.DataFrame(index=np.arange(epoch_count))

    # fixation count
    arr = get_col_array(data, col_name, ["fix_R_tStart"])
    if arr is not None:
        df_eye["fix_count"] = unique_counts_per_epoch(arr)

    # average fixation duration
    arr = get_col_array(data, col_name, ["fix_R_duration"])
    if arr is not None:
        df_eye["fix_R_duration"] = mean_per_epoch(arr)

    # fixation pupil average
    arr = get_col_array(data, col_name, ["fix_R_pupilAvg"])
    if arr is not None:
        df_eye["fix_R_pupilAvg"] = mean_per_epoch(arr)

    # saccade count
    arr = get_col_array(data, col_name, ["sacc_R_tStart"])
    if arr is not None:
        df_eye["sacc_count"] = unique_counts_per_epoch(arr)

    # saccade duration
    arr = get_col_array(data, col_name, ["sacc_R_duration"])
    if arr is not None:
        df_eye["sacc_R_duration"] = mean_per_epoch(arr)

    # saccade amplitude
    arr = get_col_array(data, col_name, ["sacc_R_ampDeg"])
    if arr is not None:
        df_eye["sacc_R_ampDeg"] = mean_per_epoch(arr)

    # saccade peak velocity
    arr = get_col_array(data, col_name, ["sacc_R_vPeak"])
    if arr is not None:
        df_eye["sacc_R_vPeak"] = mean_per_epoch(arr)

    # pupil (normalized)
    arr = get_col_array(data, col_name, ["blink_interp_RPupil_norm"])
    if arr is not None:
        df_eye["pupil_avg"] = mean_per_epoch(arr)

    # drop all-NaN columns if any
    df_eye = df_eye.dropna(axis=1, how="all")

    # ==========================
    # Combine EEG + eye features and save
    # ==========================
    df_data = pd.concat([df_eeg, df_eye], axis=1)
    df_data["label"] = labels
    df_data["subject_id"] = subject_id

    out_file = os.path.join(
        subject_dir,
        f"{subject_id}_{window_size}windowed_features.csv",
    )
    df_data.to_csv(out_file, index=False)
    print(f"Saved features for {subject_id} to: {out_file}")

# ==========================
# Save epoch counts summary
# ==========================
df_epoch_counts = pd.DataFrame({
    "subject_id": all_subjects,
    "n_epochs": epoch_counts,
})
summary_file = os.path.join(
    data_root,
    f"all_subjects_{window_size}_window_epoch_counts.csv",
)
df_epoch_counts.to_csv(summary_file, index=False)

In [ ]:
import os
import pandas as pd

data_root = "/gpfs1/pi/anon/mindless_reading/data"
sfreq = 256
window_seconds = 2
window_size = int(sfreq * window_seconds)

# collect subjects
all_subjects = sorted(
    d for d in os.listdir(data_root)
    if d.startswith("s") and os.path.isdir(os.path.join(data_root, d))
)

dfs = []   # store subject dfs here

for subject_id in all_subjects:
    subject_dir = os.path.join(
        data_root,
        subject_id,
        "ml_data",
        f"{window_size}window_datasets",
    )

    feature_file = os.path.join(
        subject_dir,
        f"{subject_id}_{window_size}windowed_features.csv",
    )

    if not os.path.exists(feature_file):
        print(f"Missing features for {subject_id}, skipping")
        continue

    df_sub = pd.read_csv(feature_file)

    # safety check
    if "subject_id" not in df_sub.columns:
        df_sub["subject_id"] = subject_id

    dfs.append(df_sub)

# concatenate once all subjects are processed
df = pd.concat(dfs, ignore_index=True)
# save combined dataframe
df.to_csv(os.path.join(data_root, f"all_subjects_{window_size}windowed_features.csv"), index=False)

# Group MW classifier

In [ ]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import LeaveOneGroupOut
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    roc_auc_score,
    precision_score,
)
from sklearn.base import clone

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

# ==========================
# Config & data loading
# ==========================
data_root = "/gpfs1/pi/anon/mindless_reading/data"
sfreq = 256
window_seconds = 2
window_size = int(sfreq * window_seconds)

df = pd.read_csv(
    os.path.join(data_root, f"all_subjects_{window_size}windowed_features.csv")
)
df = df.dropna(axis=0, how="any").reset_index(drop=True)

pca_variance = 0.95  # keep 95% variance; or set an int for n_components

# Features: everything except label and subject_id
feature_cols = [c for c in df.columns if c not in ["label", "subject_id"]]
X = df[feature_cols].values
y = df["label"].values
groups = df["subject_id"].values  # for LOSO

# Separate EEG and eye-tracking features
# fix_count is the first eye-feature column
idx_fix = np.where(df.columns.str.contains("fix_count"))[0][0]

X_eeg = X[:, :idx_fix]   # EEG-only features
X_eye = X[:, idx_fix:]   # Eye-only features

# ==========================
# Models
# ==========================
base_models = {
    "logreg": LogisticRegression(max_iter=1000, n_jobs=-1),
    "linear_svc": LinearSVC(),  # no probas, but we can use decision_function
    "rbf_svc": SVC(kernel="rbf", probability=True),
    "random_forest": RandomForestClassifier(
        n_estimators=200,
        max_depth=None,
        n_jobs=-1,
        random_state=42,
    ),
    "gradient_boosting": GradientBoostingClassifier(random_state=42),
    "knn": KNeighborsClassifier(n_neighbors=5),
    "mlp": MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42),
}

# ==========================
# LOSO + prediction logging
# ==========================
logo = LeaveOneGroupOut()

all_preds = []  # per-sample predictions across all folds / models / feature sets

for feature_set, X_data in zip(
    ["EEG + Eye", "EEG", "Eye"],
    [X, X_eeg, X_eye],
):
    print(f"\n========== Feature set: {feature_set} ==========")

    for model_name, base_clf in base_models.items():
        print(f"\n=== Model: {model_name} ===")

        acc_list = []
        f1_list = []
        prec_list = []
        auc_list = []

        for train_idx, test_idx in logo.split(X_data, y, groups=groups):
            subj_test = np.unique(groups[test_idx])
            assert len(subj_test) == 1  # LOSO: only one subject held out
            subj_test = subj_test[0]

            X_train, X_test = X_data[train_idx], X_data[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            # fresh clone of classifier
            clf = clone(base_clf)

            # Pipeline: StandardScaler -> PCA -> classifier
            pipe = Pipeline([
                ("scaler", StandardScaler()),
                ("pca", PCA(n_components=pca_variance)),
                ("clf", clf),
            ])

            pipe.fit(X_train, y_train)
            y_pred = pipe.predict(X_test)

            # scores for AUC
            y_scores = None
            if hasattr(pipe, "predict_proba"):
                try:
                    y_scores = pipe.predict_proba(X_test)[:, 1]
                except Exception:
                    y_scores = None
            elif hasattr(pipe, "decision_function"):
                try:
                    y_scores = pipe.decision_function(X_test)
                except Exception:
                    y_scores = None

            acc = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            prec = precision_score(y_test, y_pred, zero_division=0)

            if (y_scores is not None) and (np.unique(y_test).size == 2):
                auc = roc_auc_score(y_test, y_scores)
            else:
                auc = np.nan

            acc_list.append(acc)
            f1_list.append(f1)
            prec_list.append(prec)
            auc_list.append(auc)

            # log per-subject fold summary
            print(
                f"  Subject {subj_test}: "
                f"acc={acc:.3f}, prec={prec:.3f}, f1={f1:.3f}, "
                f"auc={auc if not np.isnan(auc) else float('nan'):.3f}"
            )

            # store per-sample predictions for this fold
            for i, idx in enumerate(test_idx):
                all_preds.append({
                    "feature_set": feature_set,
                    "model": model_name,
                    "test_subject": subj_test,
                    "sample_idx": int(idx),
                    "y_true": int(y_test[i]),
                    "y_pred": int(y_pred[i]),
                    "y_score": float(y_scores[i]) if y_scores is not None else np.nan,
                })
            
            # break  # TEMP: only do one fold for testing; REMOVE for full LOSO

        # per-model, per-feature-set mean over subjects
        print(
            f"Mean over subjects — acc={np.mean(acc_list):.3f}, "
            f"prec={np.mean(prec_list):.3f}, "
            f"f1={np.mean(f1_list):.3f}, "
            f"auc={np.nanmean(auc_list):.3f}"
        )

# ==========================
# Build DataFrame of predictions
# ==========================
df_preds = pd.DataFrame(all_preds)

results_dir = os.path.join(data_root, "ml_results")
os.makedirs(results_dir, exist_ok=True)

pred_file = os.path.join(
    results_dir,
    f"loso_predictions_{window_size}win.csv",
)
df_preds.to_csv(pred_file, index=False)
print(f"\nSaved per-sample LOSO predictions to {pred_file}")

# ==========================
# Final metrics from saved predictions (overall)
# ==========================
rows = []
for (feature_set, model_name), g in df_preds.groupby(["feature_set", "model"]):
    y_true = g["y_true"].values
    y_pred = g["y_pred"].values
    y_score = g["y_score"].values

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)

    # AUC over all samples for this (feature_set, model)
    if np.unique(y_true).size == 2 and not np.all(np.isnan(y_score)):
        auc = roc_auc_score(y_true, y_score)
    else:
        auc = np.nan

    rows.append({
        "feature_set": feature_set,
        "model": model_name,
        "accuracy": acc,
        "precision": prec,
        "f1": f1,
        "auc": auc,
        "n_samples": len(g),
    })

df_metrics = pd.DataFrame(rows)

metrics_file = os.path.join(
    results_dir,
    f"loso_metrics_{window_size}win.csv",
)
df_metrics.to_csv(metrics_file, index=False)

print(f"\nSaved aggregated metrics to {metrics_file}")
print("\n=== Overall summary ===")
print(df_metrics.sort_values(["feature_set", "accuracy"], ascending=[True, False]))

# ==========================
# Subject-level metrics
# ==========================
rows_subj = []
for (feature_set, model_name, subj), g in df_preds.groupby(
    ["feature_set", "model", "test_subject"]
):
    y_true = g["y_true"].values
    y_pred = g["y_pred"].values
    y_score = g["y_score"].values

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)

    if np.unique(y_true).size == 2 and not np.all(np.isnan(y_score)):
        auc = roc_auc_score(y_true, y_score)
    else:
        auc = np.nan

    rows_subj.append({
        "feature_set": feature_set,
        "model": model_name,
        "subject_id": subj,
        "accuracy": acc,
        "precision": prec,
        "f1": f1,
        "auc": auc,
        "n_samples": len(g),
    })

df_subject_metrics = pd.DataFrame(rows_subj)

subj_metrics_file = os.path.join(
    results_dir,
    f"loso_subject_metrics_{window_size}win.csv",
)
df_subject_metrics.to_csv(subj_metrics_file, index=False)

print(f"\nSaved subject-level metrics to {subj_metrics_file}")


# Feature importance

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    roc_auc_score,
    precision_score,
)
from sklearn.base import clone

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

# ==========================
# Config & data loading
# ==========================
data_root = "/gpfs1/pi/anon/mindless_reading/data"
sfreq = 256
window_seconds = 2
window_size = int(sfreq * window_seconds)

df = pd.read_csv(
    os.path.join(data_root, f"all_subjects_{window_size}windowed_features.csv")
)
df = df.dropna(axis=0, how="any").reset_index(drop=True)

pca_variance = 0.95  # keep 95% variance; or set an int for n_components

# Features: everything except label and subject_id
feature_cols = [c for c in df.columns if c not in ["label", "subject_id"]]
X = df[feature_cols].values
y = df["label"].values
groups = df["subject_id"].values  # for LOSO

# Train the random forest classifier on all data for feature importance
# (This is just for feature importance visualization, not for evaluation)
X_train_scaled = StandardScaler().fit_transform(X)
pca = PCA(n_components=pca_variance)
X_train_pca = pca.fit_transform(X_train_scaled)

rf_clf = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    n_jobs=-1,
    random_state=42,
)
rf_clf.fit(X_train_pca, y)

# Get feature importances
importances = rf_clf.feature_importances_
# Sort indices by importance
indices = np.argsort(importances)[::-1]

# Get the PCA component names
n_components = X_train_pca.shape[1]
pca_feature_names = [f"PC{i+1}" for i in range(n_components)]

# Plot feature importance for top 20 components
n_features_to_plot = min(20, n_components)
plt.figure(figsize=(10, 6))
plt.title("Feature Importance - Random Forest (Top 20 PCA Components)", fontsize=14, fontweight='bold')
plt.bar(range(n_features_to_plot), importances[indices[:n_features_to_plot]], align='center')
plt.xticks(range(n_features_to_plot), [pca_feature_names[i] for i in indices[:n_features_to_plot]], rotation=45, ha='right')
plt.xlabel("PCA Components", fontsize=12)
plt.ylabel("Importance", fontsize=12)
plt.tight_layout()
plt.show()

# Also print the original feature contributions (by looking at PCA loadings)
print("\n=== Top Contributing Original Features to Most Important PCA Components ===")
for pc_idx in indices[:5]:
    print(f"\n{pca_feature_names[pc_idx]} (Importance: {importances[pc_idx]:.4f})")
    loadings = np.abs(pca.components_[pc_idx])
    top_feat_idx = np.argsort(loadings)[::-1][:5]
    for feat_idx in top_feat_idx:
        print(f"  {feature_cols[feat_idx]}: {loadings[feat_idx]:.4f}")

# Sanity check

## Randomized labels

In [ ]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import LeaveOneGroupOut
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    roc_auc_score,
    precision_score,
)
from sklearn.ensemble import RandomForestClassifier

# ==========================
# Config & data loading
# ==========================
data_root = "/gpfs1/pi/anon/mindless_reading/data"
sfreq = 256
window_seconds = 2
window_size = int(sfreq * window_seconds)

df = pd.read_csv(
    os.path.join(data_root, f"all_subjects_{window_size}windowed_features.csv")
)
df = df.dropna(axis=0, how="any").reset_index(drop=True)

pca_variance = 0.95  # keep 95% variance

# ==========================
# Features & groups
# ==========================
feature_cols = [c for c in df.columns if c not in ["label", "subject_id"]]
X = df[feature_cols].values
y = df["label"].values
groups = df["subject_id"].values  # for LOSO

# ==========================
# PERMUTE LABELS (sanity test)
# ==========================
rng = np.random.default_rng(42)
y = rng.permutation(y)

# ==========================
# Separate EEG & Eye features
# ==========================
idx_fix = np.where(df.columns.str.contains("fix_count"))[0][0]

X_eeg = X[:, :idx_fix]
X_eye = X[:, idx_fix:]

# ==========================
# Random Forest Model
# ==========================
rf_clf = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    n_jobs=-1,
    random_state=42,
)

# ==========================
# LOSO CV + logging
# ==========================
logo = LeaveOneGroupOut()

all_preds = []

for feature_set, X_data in zip(
    ["EEG + Eye", "EEG", "Eye"],
    [X, X_eeg, X_eye],
):
    print(f"\n========== Feature set: {feature_set} (PERMUTED LABELS) ==========")

    acc_list = []
    f1_list = []
    prec_list = []
    auc_list = []

    for train_idx, test_idx in logo.split(X_data, y, groups=groups):
        subj_test = np.unique(groups[test_idx])[0]

        X_train, X_test = X_data[train_idx], X_data[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        pipe = Pipeline([
            ("scaler", StandardScaler()),
            ("pca", PCA(n_components=pca_variance)),
            ("clf", rf_clf),
        ])

        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)

        # Probabilities → for AUC
        try:
            y_scores = pipe.predict_proba(X_test)[:, 1]
        except Exception:
            y_scores = None

        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, zero_division=0)

        auc = (
            roc_auc_score(y_test, y_scores)
            if (y_scores is not None and len(np.unique(y_test)) == 2)
            else np.nan
        )

        acc_list.append(acc)
        f1_list.append(f1)
        prec_list.append(prec)
        auc_list.append(auc)

        print(
            f"  Subject {subj_test}: "
            f"acc={acc:.3f}, prec={prec:.3f}, f1={f1:.3f}, "
            f"auc={auc if not np.isnan(auc) else float('nan'):.3f}"
        )

        # store sample-level predictions
        for i, idx in enumerate(test_idx):
            all_preds.append({
                "feature_set": feature_set,
                "model": "random_forest",
                "test_subject": subj_test,
                "sample_idx": int(idx),
                "y_true": int(y_test[i]),
                "y_pred": int(y_pred[i]),
                "y_score": float(y_scores[i]) if y_scores is not None else np.nan,
            })

    print(
        f"Mean over subjects — acc={np.mean(acc_list):.3f}, "
        f"prec={np.mean(prec_list):.3f}, "
        f"f1={np.mean(f1_list):.3f}, "
        f"auc={np.nanmean(auc_list):.3f}"
    )

# ==========================
# Save predictions
# ==========================
df_preds = pd.DataFrame(all_preds)

results_dir = os.path.join(data_root, "ml_results")
os.makedirs(results_dir, exist_ok=True)

pred_file = os.path.join(
    results_dir,
    f"loso_predictions_perm_RF_{window_size}win.csv",
)
df_preds.to_csv(pred_file, index=False)
print(f"\nSaved permuted RF predictions to {pred_file}")

# ==========================
# Overall metrics
# ==========================
rows = []
for feature_set, g in df_preds.groupby("feature_set"):
    y_true = g["y_true"].values
    y_pred = g["y_pred"].values
    y_score = g["y_score"].values

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)

    auc = (
        roc_auc_score(y_true, y_score)
        if (np.unique(y_true).size == 2 and not np.all(np.isnan(y_score)))
        else np.nan
    )

    rows.append({
        "feature_set": feature_set,
        "model": "random_forest",
        "accuracy": acc,
        "precision": prec,
        "f1": f1,
        "auc": auc,
        "n_samples": len(g),
    })

df_metrics = pd.DataFrame(rows)

metrics_file = os.path.join(
    results_dir,
    f"loso_metrics_perm_RF_{window_size}win.csv",
)
df_metrics.to_csv(metrics_file, index=False)

print(f"\nSaved permuted RF metrics to {metrics_file}")
print("\n=== Summary (Permuted Labels — RF Only) ===")
print(df_metrics)

# ==========================
# Subject-level metrics
# ==========================
rows_subj = []
for (feature_set, subj), g in df_preds.groupby(["feature_set", "test_subject"]):
    y_true = g["y_true"].values
    y_pred = g["y_pred"].values
    y_score = g["y_score"].values

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)

    auc = (
        roc_auc_score(y_true, y_score)
        if (np.unique(y_true).size == 2 and not np.all(np.isnan(y_score)))
        else np.nan
    )

    rows_subj.append({
        "feature_set": feature_set,
        "model": "random_forest",
        "subject_id": subj,
        "accuracy": acc,
        "precision": prec,
        "f1": f1,
        "auc": auc,
        "n_samples": len(g),
    })

df_subject_metrics = pd.DataFrame(rows_subj)

subj_metrics_file = os.path.join(
    results_dir,
    f"loso_subject_metrics_perm_RF_{window_size}win.csv",
)
df_subject_metrics.to_csv(subj_metrics_file, index=False)

print(f"\nSaved subject-level RF perm metrics to {subj_metrics_file}")

## Randomized df

In [ ]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import LeaveOneGroupOut
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    roc_auc_score,
    precision_score,
)
from sklearn.ensemble import RandomForestClassifier

# ==========================
# Config & data loading
# ==========================
data_root = "/gpfs1/pi/anon/mindless_reading/data"
sfreq = 256
window_seconds = 2
window_size = int(sfreq * window_seconds)

df = pd.read_csv(
    os.path.join(data_root, f"all_subjects_{window_size}windowed_features.csv")
)
df = df.dropna(axis=0, how="any").reset_index(drop=True)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
pca_variance = 0.95  # keep 95% variance

# ==========================
# Features & groups
# ==========================
feature_cols = [c for c in df.columns if c not in ["label", "subject_id"]]
X = df[feature_cols].values
y = df["label"].values
groups = df["subject_id"].values  # for LOSO


# ==========================
# Random Forest Model
# ==========================
rf_clf = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    n_jobs=-1,
    random_state=42,
)

# ==========================
# LOSO CV + logging (EEG + Eye only)
# ==========================
logo = LeaveOneGroupOut()

all_preds = []

feature_set = "EEG + Eye"
X_data = X  # combined features

print(f"\n========== Feature set: {feature_set} ==========")

acc_list = []
f1_list = []
prec_list = []
auc_list = []

for train_idx, test_idx in logo.split(X_data, y, groups=groups):
    subj_test = np.unique(groups[test_idx])[0]

    X_train, X_test = X_data[train_idx], X_data[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("pca", PCA(n_components=pca_variance)),
        ("clf", rf_clf),
    ])

    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)

    # Probabilities → for AUC
    try:
        y_scores = pipe.predict_proba(X_test)[:, 1]
    except Exception:
        y_scores = None

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)

    auc = (
        roc_auc_score(y_test, y_scores)
        if (y_scores is not None and len(np.unique(y_test)) == 2)
        else np.nan
    )

    acc_list.append(acc)
    f1_list.append(f1)
    prec_list.append(prec)
    auc_list.append(auc)

    print(
        f"  Subject {subj_test}: "
        f"acc={acc:.3f}, prec={prec:.3f}, f1={f1:.3f}, "
        f"auc={auc if not np.isnan(auc) else float('nan'):.3f}"
    )

    # store sample-level predictions
    for i, idx in enumerate(test_idx):
        all_preds.append({
            "feature_set": feature_set,
            "model": "random_forest",
            "test_subject": subj_test,
            "sample_idx": int(idx),
            "y_true": int(y_test[i]),
            "y_pred": int(y_pred[i]),
            "y_score": float(y_scores[i]) if y_scores is not None else np.nan,
        })

print(
    f"Mean over subjects — acc={np.mean(acc_list):.3f}, "
    f"prec={np.mean(prec_list):.3f}, "
    f"f1={np.mean(f1_list):.3f}, "
    f"auc={np.nanmean(auc_list):.3f}"
)

# ==========================
# Save predictions
# ==========================
df_preds = pd.DataFrame(all_preds)

results_dir = os.path.join(data_root, "ml_results")
os.makedirs(results_dir, exist_ok=True)

pred_file = os.path.join(
    results_dir,
    f"loso_predictions_RF_{window_size}win.csv",
)
df_preds.to_csv(pred_file, index=False)
print(f"\nSaved RF predictions to {pred_file}")

# ==========================
# Overall metrics (EEG + Eye only)
# ==========================
rows = []
g = df_preds  # only one feature_set

y_true = g["y_true"].values
y_pred = g["y_pred"].values
y_score = g["y_score"].values

acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, zero_division=0)

auc = (
    roc_auc_score(y_true, y_score)
    if (np.unique(y_true).size == 2 and not np.all(np.isnan(y_score)))
    else np.nan
)

rows.append({
    "feature_set": feature_set,
    "model": "random_forest",
    "accuracy": acc,
    "precision": prec,
    "f1": f1,
    "auc": auc,
    "n_samples": len(g),
})

df_metrics = pd.DataFrame(rows)

metrics_file = os.path.join(
    results_dir,
    f"loso_metrics_RF_{window_size}win.csv",
)
df_metrics.to_csv(metrics_file, index=False)

print(f"\nSaved RF metrics to {metrics_file}")
print("\n=== Summary (EEG + Eye — RF Only) ===")
print(df_metrics)

# ==========================
# Subject-level metrics
# ==========================
rows_subj = []
for subj, g_subj in df_preds.groupby("test_subject"):
    y_true = g_subj["y_true"].values
    y_pred = g_subj["y_pred"].values
    y_score = g_subj["y_score"].values

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)

    auc = (
        roc_auc_score(y_true, y_score)
        if (np.unique(y_true).size == 2 and not np.all(np.isnan(y_score)))
        else np.nan
    )

    rows_subj.append({
        "feature_set": feature_set,
        "model": "random_forest",
        "subject_id": subj,
        "accuracy": acc,
        "precision": prec,
        "f1": f1,
        "auc": auc,
        "n_samples": len(g_subj),
    })

df_subject_metrics = pd.DataFrame(rows_subj)

subj_metrics_file = os.path.join(
    results_dir,
    f"loso_subject_metrics_RF_{window_size}win.csv",
)
df_subject_metrics.to_csv(subj_metrics_file, index=False)

print(f"\nSaved subject-level RF metrics to {subj_metrics_file}")


## Plot subject performance (LOSO)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
import warnings
warnings.filterwarnings(
    "ignore",
    message="When grouping with a length-1 list-like",
    category=FutureWarning
)

# ---------------------------
# Config
# ---------------------------
file = "/gpfs1/pi/anon/mindless_reading/data/ml_results/loso_subject_metrics_512win.csv"
output_pdf = "loso_subject_metric_histograms_512win.pdf"

# ---------------------------
# Load data
# ---------------------------
df = pd.read_csv(file)

metrics = ["accuracy", "precision", "f1", "auc"]
metric_pretty = {
    "accuracy": "Accuracy",
    "precision": "Precision",
    "f1": "F1-score",
    "auc": "ROC-AUC",
}

feature_order = ["EEG + Eye", "EEG", "Eye"]
feature_colors = {
    "EEG + Eye": "tab:blue",
    "EEG": "tab:orange",
    "Eye": "tab:green",
}

models = df["model"].unique()

sns.set(style="whitegrid")

# ---------------------------
# Create PDF & write pages
# ---------------------------
with PdfPages(output_pdf) as pdf:

    for model_name in models:
        df_model = df[df["model"] == model_name].copy()

        # drop rows with all metrics NaN (just in case)
        if df_model[metrics].isna().all(axis=1).all():
            print(f"Skipping model {model_name}: no valid metrics.")
            continue

        fig, axes = plt.subplots(1, 4, figsize=(20, 4), sharey=False)
        fig.suptitle(f"{model_name} — subject-level metrics", fontsize=14)

        for ax, metric in zip(axes, metrics):
            df_m = df_model[~df_model[metric].isna()].copy()
            if df_m.empty:
                ax.set_visible(False)
                continue

            # seaborn histogram with hue = feature_set
            sns.histplot(
                data=df_m[df_m["feature_set"].isin(feature_order)],
                x=metric,
                hue="feature_set",
                hue_order=feature_order,
                palette=feature_colors,
                bins=10,
                stat="count",
                common_norm=False,
                alpha=0.4,
                edgecolor="black",
                ax=ax,
                element="poly",
            )

            # Add mean lines per feature_set
            for feat in feature_order:
                vals = df_m.loc[df_m["feature_set"] == feat, metric].values
                if len(vals) == 0:
                    continue
                mean_val = np.mean(vals)
                ax.axvline(
                    mean_val,
                    linestyle="--",
                    linewidth=2,
                    color=feature_colors.get(feat, "black"),
                    label=f"{feat} mean={mean_val:.3f}",
                )

            ax.set_title(metric_pretty.get(metric, metric))
            ax.set_xlabel(metric_pretty.get(metric, metric))
            ax.set_ylabel("Number of subjects")

            # Avoid duplicate legend entries from histplot + mean lines
            handles, labels = ax.get_legend_handles_labels()
            # Deduplicate while preserving order
            seen = set()
            uniq_handles = []
            uniq_labels = []
            for h, l in zip(handles, labels):
                if l not in seen:
                    seen.add(l)
                    uniq_handles.append(h)
                    uniq_labels.append(l)
            ax.legend(uniq_handles, uniq_labels, fontsize=8)

        plt.tight_layout(rect=[0, 0, 1, 0.95])
        pdf.savefig(fig)
        plt.close(fig)

print(f"\nSaved all plots to:\n  {output_pdf}")


## PCA

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# ==========================
# Config & data loading
# ==========================
data_root = "/gpfs1/pi/anon/mindless_reading/data"
sfreq = 256
window_seconds = 2
window_size = int(sfreq * window_seconds)

df = pd.read_csv(
    os.path.join(data_root, f"all_subjects_{window_size}windowed_features.csv")
)
df = df.dropna(axis=0, how="any").reset_index(drop=True)
pca_variance = 0.95  # keep 95% variance

# ==========================
# Features & groups
# ==========================
feature_cols = [c for c in df.columns if c not in ["label", "subject_id"]]
X = df[feature_cols].values
y = df["label"].values
groups = df["subject_id"].values  # for LOSO

# Standardize + PCA (fit on all data for visualization)
scaler = StandardScaler()
Xz = scaler.fit_transform(X)

pca = PCA(n_components=pca_variance, random_state=42)
Z = pca.fit_transform(Xz)   # PC scores: shape (n_samples, n_components)

labels = np.asarray(y)
classes = np.unique(labels)

pc_x = 32  # PC33 (0-based index)
pc_y = 11  # PC12

labels = np.asarray(y)
classes = np.unique(labels)

# robust zoom limits (ignore extreme outliers)
xlo, xhi = np.percentile(Z[:, pc_x], [1, 99])
ylo, yhi = np.percentile(Z[:, pc_y], [1, 99])

plt.figure(figsize=(7, 6))
for c in classes:
    idx = labels == c
    plt.scatter(
        Z[idx, pc_x],
        Z[idx, pc_y],
        s=10,
        alpha=0.35,
        label=f"class {c} (n={idx.sum()})"
    )

plt.xlim(xlo, xhi)
plt.ylim(ylo, yhi)

plt.xlabel(f"PC{pc_x+1} ({pca.explained_variance_ratio_[pc_x]*100:.2f}% var)")
plt.ylabel(f"PC{pc_y+1} ({pca.explained_variance_ratio_[pc_y]*100:.2f}% var)")
plt.title(f"PCA: PC{pc_x+1} vs PC{pc_y+1}")
plt.legend()
plt.tight_layout()
plt.show()


pc_x = 0  
pc_y = 1  

labels = np.asarray(y)
classes = np.unique(labels)

# robust zoom limits (ignore extreme outliers)
xlo, xhi = np.percentile(Z[:, pc_x], [1, 99])
ylo, yhi = np.percentile(Z[:, pc_y], [1, 99])

plt.figure(figsize=(7, 6))
for c in classes:
    idx = labels == c
    plt.scatter(
        Z[idx, pc_x],
        Z[idx, pc_y],
        s=10,
        alpha=0.35,
        label=f"class {c} (n={idx.sum()})"
    )

plt.xlim(xlo, xhi)
plt.ylim(ylo, yhi)

plt.xlabel(f"PC{pc_x+1} ({pca.explained_variance_ratio_[pc_x]*100:.2f}% var)")
plt.ylabel(f"PC{pc_y+1} ({pca.explained_variance_ratio_[pc_y]*100:.2f}% var)")
plt.title(f"PCA: PC{pc_x+1} vs PC{pc_y+1}")
plt.legend()
plt.tight_layout()
plt.show()

## Plot EEG features

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mne
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

# ==========================
# Config & data loading
# ==========================
data_root = "/gpfs1/pi/anon/mindless_reading/data"
sfreq = 256
window_seconds = 2
window_size = int(sfreq * window_seconds)

df = pd.read_csv(os.path.join(data_root, f"all_subjects_{window_size}windowed_features.csv"))
df = df.dropna(axis=0, how="any").reset_index(drop=True)

n_ch = 64
n_band = 8
n_eeg = n_ch * n_band

# EEG only (assumes first 512 columns are EEG chan_band features)
eeg_cols = df.columns[:n_eeg]
X = df.loc[:, eeg_cols].to_numpy(dtype=float)
y = df["label"].to_numpy()

# ==========================
# Parse channel & band names
# ==========================
ch_names, band_names = [], []
for c in eeg_cols:
    ch, band = c.split("_", 1)
    if ch not in ch_names:
        ch_names.append(ch)
    if band not in band_names:
        band_names.append(band)

assert len(ch_names) == n_ch, f"Expected {n_ch} channels, got {len(ch_names)}"
assert len(band_names) == n_band, f"Expected {n_band} bands, got {len(band_names)}"

# (Optional) enforce canonical band order if you want:
canonical_band_order = ["theta1","theta2","alpha1","alpha2","beta1","beta2","gamma1","gamma2"]
if set(canonical_band_order) == set(band_names):
    band_names = canonical_band_order  # reorder labels ONLY (see note below)

# ==========================
# Compute class difference: (class1 - class0)
# ==========================
X3 = X.reshape(-1, n_ch, n_band)  # (n_samples, 64, 8)

X0 = X3[y == 0]
X1 = X3[y == 1]

mean_diff = X1.mean(axis=0) - X0.mean(axis=0)  # (64, 8)

# ==========================
# Topomaps per band
# ==========================
rename_map = {"Afz": "AFz"}
ch_names_fixed = [rename_map.get(ch, ch) for ch in ch_names]
info = mne.create_info(ch_names=ch_names_fixed, sfreq=sfreq, ch_types="eeg")
montage = mne.channels.make_standard_montage("biosemi64")
info.set_montage(montage)

# robust symmetric color limits
# vmax = np.percentile(np.abs(mean_diff), 95)
# vlim = (-vmax, vmax)

# def add_colorbar(fig, vmin, vmax, label):
#     sm = ScalarMappable(norm=Normalize(vmin=vmin, vmax=vmax), cmap="RdBu_r")
#     sm.set_array([])
#     cbar = fig.colorbar(
#         sm, ax=fig.axes, orientation="vertical",
#         fraction=0.02, pad=0.02
#     )
#     cbar.set_label(label)

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
for b, band in enumerate(band_names):
    ax = axes.flat[b]
    im, _ = mne.viz.plot_topomap(
        X0.mean(axis=0)[:, b],
        info,
        axes=axes.flat[b],
        show=False,
        cmap="RdBu_r",
        contours=0
    )
    axes.flat[b].set_title(band)
    fig.colorbar(im, ax=axes.flat[b], fraction=0.046, pad=0.04)

plt.suptitle("Class 0: Normal Reading", y=1.02)
# add_colorbar(fig, vmin, vmax, "EEG band power")
plt.show()

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
for b, band in enumerate(band_names):
    ax = axes.flat[b]
    im, _ = mne.viz.plot_topomap(
        X1.mean(axis=0)[:, b],
        info,
        axes=axes.flat[b],
        show=False,
        cmap="RdBu_r",
        contours=0
    )
    axes.flat[b].set_title(band)
    fig.colorbar(im, ax=axes.flat[b], fraction=0.046, pad=0.04)

plt.suptitle("Class 1: Mind-Wandering", y=1.02)
# add_colorbar(fig, vmin, vmax, "EEG band power")
plt.show()

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
for b, band in enumerate(band_names):
    ax = axes.flat[b]
    im, _ = mne.viz.plot_topomap(
        mean_diff[:, b],
        info,
        axes=axes.flat[b],
        show=False,
        cmap="RdBu_r",
        contours=0
    )
    axes.flat[b].set_title(band)
    fig.colorbar(im, ax=axes.flat[b], fraction=0.046, pad=0.04)

plt.suptitle("Topographic difference (Class 1 − Class 0)", y=1.02)
# add_colorbar(fig, vmin, vmax, "Δ EEG band power")
plt.show()






## Inner EEG features

In [ ]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import LeaveOneGroupOut
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    roc_auc_score,
    precision_score,
)
from sklearn.ensemble import RandomForestClassifier

# ==========================
# Config & data loading
# ==========================
data_root = "/gpfs1/pi/anon/mindless_reading/data"
sfreq = 256
window_seconds = 2
window_size = int(sfreq * window_seconds)

df = pd.read_csv(
    os.path.join(data_root, f"all_subjects_{window_size}windowed_features.csv")
)
df = df.dropna(axis=0, how="any").reset_index(drop=True)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
pca_variance = 0.95  # keep 95% variance

# drop outer EEG channels
outer_channels_biosemi64 = [
    # frontal
    "Fp1", "Fp2", "AF7", "AF8", "AF3", "AF4", "Afz",
    # left temporal
    "FT7", "F7", "T7", "TP7",
    # right temporal
    "FT8", "F8", "T8", "TP8",
    # posterior
    "PO7", "PO8", "O1", "Oz", "O2",
    # inferior parietal
    "P7", "P8",
]
# build tuple of channel prefixes like "Fp1_", "AF7_", etc.
outer_prefixes = tuple(f"{ch}_" for ch in outer_channels_biosemi64)
# columns to drop
cols_to_drop = [c for c in df.columns if c.startswith(outer_prefixes)]
df = df.drop(columns=cols_to_drop)

# ==========================
# Features & groups
# ==========================
feature_cols = [c for c in df.columns if c not in ["label", "subject_id"]]
X = df[feature_cols].values
y = df["label"].values
groups = df["subject_id"].values  # for LOSO

# Separate EEG and eye-tracking features
# fix_count is the first eye-feature column
idx_fix = np.where(df.columns.str.contains("fix_count"))[0][0]

X_eeg = X[:, :idx_fix]   # EEG-only features
X_eye = X[:, idx_fix:]   # Eye-only features

# ==========================
# Random Forest Model
# ==========================
rf_clf = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    n_jobs=-1,
    random_state=42,
)

# ==========================
# LOSO CV + logging (EEG + Eye only)
# ==========================
logo = LeaveOneGroupOut()

for feature_set, X_data in zip(
    ["EEG + Eye", "EEG", "Eye"],
    [X, X_eeg, X_eye],
):
    print(f"\n========== Feature set: {feature_set} ==========")
    acc_list = []
    f1_list = []
    prec_list = []
    auc_list = []
    for train_idx, test_idx in logo.split(X_data, y, groups=groups):
        subj_test = np.unique(groups[test_idx])[0]

        X_train, X_test = X_data[train_idx], X_data[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        pipe = Pipeline([
            ("scaler", StandardScaler()),
            ("pca", PCA(n_components=pca_variance)),
            ("clf", rf_clf),
        ])

        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)

        # Probabilities → for AUC
        try:
            y_scores = pipe.predict_proba(X_test)[:, 1]
        except Exception:
            y_scores = None

        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, zero_division=0)

        auc = (
            roc_auc_score(y_test, y_scores)
            if (y_scores is not None and len(np.unique(y_test)) == 2)
            else np.nan
        )

        acc_list.append(acc)
        f1_list.append(f1)
        prec_list.append(prec)
        auc_list.append(auc)

        print(
            f"  Subject {subj_test}: "
            f"acc={acc:.3f}, prec={prec:.3f}, f1={f1:.3f}, "
            f"auc={auc if not np.isnan(auc) else float('nan'):.3f}"
        )

        # # store sample-level predictions
        # for i, idx in enumerate(test_idx):
        #     all_preds.append({
        #         "feature_set": feature_set,
        #         "model": "random_forest",
        #         "test_subject": subj_test,
        #         "sample_idx": int(idx),
        #         "y_true": int(y_test[i]),
        #         "y_pred": int(y_pred[i]),
        #         "y_score": float(y_scores[i]) if y_scores is not None else np.nan,
        #     })

    print(
        f"Mean over subjects — acc={np.mean(acc_list):.3f}, "
        f"prec={np.mean(prec_list):.3f}, "
        f"f1={np.mean(f1_list):.3f}, "
        f"auc={np.nanmean(auc_list):.3f}"
    )

# # ==========================
# # Save predictions
# # ==========================
# df_preds = pd.DataFrame(all_preds)

# results_dir = os.path.join(data_root, "ml_results")
# os.makedirs(results_dir, exist_ok=True)

# pred_file = os.path.join(
#     results_dir,
#     f"loso_predictions_RF_{window_size}win.csv",
# )
# df_preds.to_csv(pred_file, index=False)
# print(f"\nSaved RF predictions to {pred_file}")

# # ==========================
# # Overall metrics (EEG + Eye only)
# # ==========================
# rows = []
# g = df_preds  # only one feature_set

# y_true = g["y_true"].values
# y_pred = g["y_pred"].values
# y_score = g["y_score"].values

# acc = accuracy_score(y_true, y_pred)
# f1 = f1_score(y_true, y_pred)
# prec = precision_score(y_true, y_pred, zero_division=0)

# auc = (
#     roc_auc_score(y_true, y_score)
#     if (np.unique(y_true).size == 2 and not np.all(np.isnan(y_score)))
#     else np.nan
# )

# rows.append({
#     "feature_set": feature_set,
#     "model": "random_forest",
#     "accuracy": acc,
#     "precision": prec,
#     "f1": f1,
#     "auc": auc,
#     "n_samples": len(g),
# })

# df_metrics = pd.DataFrame(rows)

# metrics_file = os.path.join(
#     results_dir,
#     f"loso_metrics_RF_{window_size}win.csv",
# )
# df_metrics.to_csv(metrics_file, index=False)

# print(f"\nSaved RF metrics to {metrics_file}")
# print("\n=== Summary (EEG + Eye — RF Only) ===")
# print(df_metrics)

# # ==========================
# # Subject-level metrics
# # ==========================
# rows_subj = []
# for subj, g_subj in df_preds.groupby("test_subject"):
#     y_true = g_subj["y_true"].values
#     y_pred = g_subj["y_pred"].values
#     y_score = g_subj["y_score"].values

#     acc = accuracy_score(y_true, y_pred)
#     f1 = f1_score(y_true, y_pred)
#     prec = precision_score(y_true, y_pred, zero_division=0)

#     auc = (
#         roc_auc_score(y_true, y_score)
#         if (np.unique(y_true).size == 2 and not np.all(np.isnan(y_score)))
#         else np.nan
#     )

#     rows_subj.append({
#         "feature_set": feature_set,
#         "model": "random_forest",
#         "subject_id": subj,
#         "accuracy": acc,
#         "precision": prec,
#         "f1": f1,
#         "auc": auc,
#         "n_samples": len(g_subj),
#     })

# df_subject_metrics = pd.DataFrame(rows_subj)

# subj_metrics_file = os.path.join(
#     results_dir,
#     f"loso_subject_metrics_RF_{window_size}win.csv",
# )
# df_subject_metrics.to_csv(subj_metrics_file, index=False)

# print(f"\nSaved subject-level RF metrics to {subj_metrics_file}")